# Implementación del algoritmo de Newton-Raphson

## 0 - Aplicación de Newton-Raphson al sistema estudiado

### Paso 1 : Importación de las librerias

In [10]:
import numpy as np # type: ignore
from numpy import * # type: ignore

### Paso 2: Declaración de las variables del sistema (impedencia, potencias activas y reactivas demandadas, tensione sy angulos de inicio)

In [11]:
#Impedencia por unidad de largo de las lineas
r_linea = 0.099 ;
x_linea = 0.156 ;
Z_linea = complex(r_linea, x_linea) ;

#Largo de las lineas
L21A=10;
L1A2A=15;
L2A3A=20;
L3A2B=15;
L2B1B=30;
L1B2=10;

#Calculo de impedencias de linea
Z21A = Z_linea*L21A;
Z1A2A = Z_linea*L1A2A ;
Z2A3A = Z_linea*L2A3A;
Z3A2B = Z_linea*L3A2B;
Z2B1B = Z_linea*L2B1B;
Z1B2 = Z_linea*L1B2;

#Impedencia del transformador
ZT = complex(0,0.0012); 

#Calculo de los componentes de la matriz de admitancia
Y11 = 1/ZT ;
Y12 = -1/ZT;
Y22 = 1/ZT + 1/Z21A + 1/Z1B2;
Y21A = -1/Z21A ;
Y21B = -1/Z1B2;
Y1A1A = 6 ; #COMPLETAR
Y1A2A = 7; #COMPLETAR
Y2A2A = 8; #COMPLETAR
Y2A3A = 9; #COMPLETAR
Y3A3A = 10; #COMPLETAR
Y3A2B = 11; #COMPLETAR
Y2B2B = 12; #COMPLETAR
Y2B1B = 13; #COMPLETAR
Y1B1B = 14; #COMPLETAR

#Declaración de la matriz de admitancia
Ybus = np.matrix([[Y11, Y12, 0, 0, 0, 0, 0] , [Y12, Y22, Y21A, 0, 0, Y21B, 0] , [0, Y21A, Y1A1A, Y1A2A, 0, 0, 0] , [0, 0, Y1A2A, Y2A2A, Y2A3A, 0, 0] , [0, 0, 0 ,Y2A3A ,Y3A3A ,0 ,Y3A2B], [0, Y21B, 0 ,0 ,0 ,Y1B1B, Y2B1B] , [0 ,0 ,0 ,0 ,Y3A2B ,Y2B1B, Y2B2B]]) ; print(Ybus)

#Inicialización de las tensiones y angulos
V = array();
delta = array();

#Definición de las potencias activas y reactivas demandadas en cada barra
PQ = array();


#BORRAR ESO CUANDO SE ACABA
# Ejemplo de uso:
# Definir la matriz de admitancia Ybus del sistema
#Ybus = np.array([[complex(5, -20), complex(-10, 20)],
 #                [complex(-3, 20), complex(10, -20)]])

# Inicializar las tensiones y ángulos
#V = np.array([1.0, 1.0])
#delta = np.array([0.0, 0.0])

# Definir las potencias activas y reactivas demandadas (P, Q)
#PQ = np.array([[0.0, 0.0],
 #              [-0.5, -0.3]])




(0.099+0.156j)
[[ 0.        -8333.33333333j  0.        +8333.33333333j
   0.           +0.j          0.           +0.j
   0.           +0.j          0.           +0.j
   0.           +0.j        ]
 [ 0.        +8333.33333333j  0.29000791-8333.79031549j
   4.           +0.j          0.           +0.j
   0.           +0.j          5.           +0.j
   0.           +0.j        ]
 [ 0.           +0.j          4.           +0.j
   6.           +0.j          7.           +0.j
   0.           +0.j          0.           +0.j
   0.           +0.j        ]
 [ 0.           +0.j          0.           +0.j
   7.           +0.j          8.           +0.j
   9.           +0.j          0.           +0.j
   0.           +0.j        ]
 [ 0.           +0.j          0.           +0.j
   0.           +0.j          9.           +0.j
  10.           +0.j          0.           +0.j
  11.           +0.j        ]
 [ 0.           +0.j          5.           +0.j
   0.           +0.j          0.           +0.j
   

TypeError: array() missing required argument 'object' (pos 0)

### Paso 3: Ejecución y resultados del algoritmo de Newton-Raphson

In [ ]:
# Ejecutar el método de Newton-Raphson
V, delta = newton_raphson(Ybus, V, delta, PQ)

# Resultados
print('Tensiones en las barras (pu):', V)
print('Ángulos de las tensiones (rad):', delta)

## 1 - Función de calculo de la jacobiana

In [ ]:

def calculate_jacobian(Ybus, V, delta, PQ):
    n = len(V)
    J = np.zeros((2 * n, 2 * n))

    for i in range(n):
        for j in range(n):
            if i == j:
                for k in range(n):
                    if k != i:
                        J[i, i] += V[k] * (Ybus[i, k].real * np.cos(delta[i] - delta[k]) +
                                           Ybus[i, k].imag * np.sin(delta[i] - delta[k]))
                        J[i + n, i] += V[k] * (Ybus[i, k].real * np.sin(delta[i] - delta[k]) -
                                               Ybus[i, k].imag * np.cos(delta[i] - delta[k]))
                J[i, i] += V[i] * Ybus[i, i].real
                J[i + n, i] -= V[i] * Ybus[i, i].imag
            else:
                J[i, j] = V[i] * V[j] * (Ybus[i, j].real * np.cos(delta[i] - delta[j]) +
                                         Ybus[i, j].imag * np.sin(delta[i] - delta[j]))
                J[i + n, j] = V[i] * V[j] * (Ybus[i, j].real * np.sin(delta[i] - delta[j]) -
                                             Ybus[i, j].imag * np.cos(delta[i] - delta[j]))
                J[i, j + n] = V[j] * (Ybus[i, j].real * np.cos(delta[i] - delta[j]) +
                                      Ybus[i, j].imag * np.sin(delta[i] - delta[j]))
                J[i + n, j + n] = V[j] * (Ybus[i, j].real * np.sin(delta[i] - delta[j]) -
                                          Ybus[i, j].imag * np.cos(delta[i] - delta[j]))

    return J


## 2 - Funcion de calculo de la diferencia entre potencia inyecta y potencia calculada (función de error)

In [ ]:
def power_mismatch(Ybus, V, delta, PQ):
    n = len(V)
    P = np.zeros(n)
    Q = np.zeros(n)

    for i in range(n):
        for j in range(n):
            P[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(delta[i] - delta[j]) +
                                   Ybus[i, j].imag * np.sin(delta[i] - delta[j]))
            Q[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(delta[i] - delta[j]) -
                                   Ybus[i, j].imag * np.cos(delta[i] - delta[j]))

    dP = PQ[:, 0] - P
    dQ = PQ[:, 1] - Q
    return np.concatenate([dP, dQ])


## 3 - Funcion principal de iteraciones del algoritmo

In [ ]:
def newton_raphson(Ybus, V, delta, PQ, tol=1e-6, max_iter=100):
    n = len(V)
    for iteration in range(max_iter):
        mismatch = power_mismatch(Ybus, V, delta, PQ)
        if np.linalg.norm(mismatch) < tol:
            print(f'Tolerancia alcanzada después {iteration} iteraciones.')
            break

        J = calculate_jacobian(Ybus, V, delta, PQ)
        update = np.linalg.solve(J, mismatch)
        delta[:n] += update[:n]
        V[1:] += update[n:]

    return V, delta
